#### Trabalho 1 - Caracterizando redes com o graph-tool - Entrega e apresentação: 16/9 - Pós-Graduação (apenas entrega): 21/10
Nesta tarefa você deve instalar e se familiarizar como graph-tool que é um módulo Python (eficiente) para manipular e analisar redes, ou algum outro software para manipular redes, como o NetworkX ou o Gephi que é uma excelente ferramenta para desenhar redes. Você deve escolher ao menos quatro redes disponíveis nos repositórios abaixo ou em outros repositórios (e não apenas as redes disponíveis no graph-tool), e caracterizá-las utilizando diferentes métricas, como grau, distância, tamanho das componentes conexas, e outras (você pode escolher outras). Para cada métrica analisada, calcule as seguintes estatísticas: 

* máximo
* mínimo
* média
* mediana
* desvio padrão
* e distribuição empírica (faça um gráfico).

### Dataset 0 - protein network
* retirado dos datasets do [livro do Barabási](http://networksciencebook.com/translations/en/resources/data.html)

*Network representing the protein-protein interactions in yeast. Each node represents a protein, and they are connected if they physically interact within the cell.*

Original data: [Yu, H., Braun, P., Yam, M. A., Lemmens, I., Venkatesan, K., Sahalie, J., ... & Vidal, M. (2008). High-quality binary protein interaction map of the yeast interactome network. Science, 322(5898), 104-110.](http://interactome.dfci.harvard.edu/S_cerevisiae/index.php?page=download)

In [2]:
import sys
import pandas as pd
import networkx as nx
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from math import log

plt.style.use("seaborn")

In [4]:
# Carregando o dataset
df = pd.read_csv("data/protein.edgelist.txt", sep="\t")
df.columns = ("source", "target")
G = nx.from_pandas_edgelist(df)

In [10]:
len(G.nodes), len(G.edges)

(2017, 2929)

In [6]:
# Carregando dataframe de graus
degrees = pd.DataFrame.from_dict(dict(G.degree), orient="index")
degrees.columns = ["degree"]
degrees.index.name = "node"
degrees.sort_values(by="degree", inplace=True)

### Métricas para grau

In [126]:
# 5 graus de maior tamanho
degrees["degree"].nlargest()

node
1356    91
1400    82
1637    81
1017    52
528     46
Name: degree, dtype: int64

In [130]:
# 5 graus de menor tamanho
degrees["degree"].drop_duplicates().nsmallest(keep="first")

node
1       1
1171    2
698     3
301     4
1919    5
Name: degree, dtype: int64

In [16]:
# 5 que mais ocorrem
degrees["degree"].value_counts()[:20]

1     974
2     414
3     204
4     135
5      67
6      56
7      35
8      35
9      19
10     12
11     11
12     10
15      6
14      4
20      4
18      4
29      4
13      4
21      3
26      2
Name: degree, dtype: int64

In [11]:
degrees["degree"].mean()

2.9043133366385723

In [12]:
degrees["degree"].median()

2.0

In [1]:
def plot(data,filename,dataset):
    # Adapted from 
    # https://gist.github.com/yamaguchiyuto/504eb5482fc73f046f6b
   
    """ Plot Distribution """
    plt.plot(range(len(data)),data,'bo')
    plt.yscale('log')
    plt.xscale('log')
    plt.ylabel('Freq')
    plt.xlabel(f'Degree - {dataset} dataset')
    plt.savefig(filename + '_' + dataset + '_distribution.png')
    plt.clf()

    """ Plot CDF """
    s = float(data.sum())
    cdf = data.cumsum(0)/s
    plt.plot(range(len(cdf)),cdf,'bo')
    plt.xscale('log')
    plt.ylim([0,1])
    plt.ylabel('CDF')
    plt.xlabel(f'Degree - {dataset} dataset')
    plt.savefig(filename + '_' + dataset + '_cdf.png')
    plt.clf()

    """ Plot CCDF """
    ccdf = 1-cdf
    plt.plot(range(len(ccdf)),ccdf,'bo')
    plt.xscale('log')
    plt.yscale('log')
    plt.ylim([0,1])
    plt.ylabel('CCDF')
    plt.xlabel(f'Degree - {dataset} dataset')
    plt.savefig(filename + '_' + dataset + '_ccdf.png')
    plt.clf()
    
edgelist_file = "data/protein.edgelist.txt"

""" Load graph """
df = pd.read_csv(edgelist_file, sep="\t")
df.columns = ("source", "target")
G = nx.from_pandas_edgelist(df)

""" To sparse adjacency matrix """
M = nx.to_scipy_sparse_matrix(G)

indegrees = M.sum(0).A[0]
outdegrees = M.sum(1).T.A[0]
indegree_distribution = np.bincount(indegrees)
outdegree_distribution = np.bincount(outdegrees)

plot(indegree_distribution, edgelist_file, 'protein',)

NameError: name 'pd' is not defined